In [1]:
import config
from assistants.json_assistants import JSONAssistantYA, JSONAssistantSber, JSONAssistantGPT, JSONAssistantMistralAI
from assistants.simple_assistants import SimpleAssistantSber, SimpleAssistantGPT

from pydantic import BaseModel, Field
from typing import List, Any, Optional, Dict, Tuple

import pandas as pd
import re
import os
import json

from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [2]:
with open('./data/terms&defs.txt', 'r', encoding='utf-8') as f:
    glossary = f.read()

prompt = """Создайте резюме документа на русском языке. Используйте перечень Сокращения, Терминов и определений и Подразделений из контекста Glossary.

Glossary:
{glossary}

"""

prompt_init = """Верните список используемых терминов и аббревиатур из документа пользователя.

"""

class Term(BaseModel):
    term: Optional[str] = Field(description="Термин или аббревиатура.")

class Terms(BaseModel):
    """Верните список используемых терминов и аббревиатур из документа пользователя."""

    terms: Optional[List[Term]] = Field(description="Список терминов или аббревиатур.")

In [3]:
def extract_terms_definitions(text: str) -> List[Dict[str, str]]:
    """
    Extracts terms and their definitions from a structured text document.

    Args:
        text (str): The input text containing terms and definitions in the following format:
                    ##Term: <Term>
                    ##Definition: <Definition>

    Returns:
        List[Dict[str, str]]: A list of dictionaries with 'term' and 'definition' keys.
    """
    # Define the regular expression pattern
    pattern = r'##Term:\s*(.*?)\s*##Definition:\s*(.*?)(?=##Term:|$)'

    # Use re.findall to extract all matches
    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)

    # Process matches to create a list of dictionaries
    terms_definitions = []
    for term, definition in matches:
        # Clean up the extracted term and definition by stripping whitespace
        term_clean = term.strip()
        definition_clean = definition.strip()

        # Append to the list as a dictionary
        terms_definitions.append({
            'term': term_clean,
            'definition': definition_clean
        })

    return terms_definitions

In [4]:
def dicts_to_documents(terms_definitions: List[Dict[str, str]]) -> List[Document]:
    """
    Transforms a list of dictionaries with 'term' and 'definition' into a list of LangChain Documents.

    Args:
        terms_definitions (List[Dict[str, str]]): List of dictionaries containing 'term' and 'definition'.

    Returns:
        List[Document]: List of LangChain Document objects.
    """
    documents = []
    for entry in terms_definitions:
        term = entry.get('term', '').strip()
        definition = entry.get('definition', '').strip()
        
        # Create a Document with definition as content and term as metadata
        doc = Document(
            page_content=f'{term}: {definition}',
            metadata={"term": term}
        )
        documents.append(doc)
    
    return documents


In [5]:
t_and_d = extract_terms_definitions(glossary).copy()

In [6]:
t_d_docs = dicts_to_documents(t_and_d)
embeddings = HuggingFaceEmbeddings(model_name='models/multilingual-e5-large')
t_d_vs = FAISS.from_documents(t_d_docs, embeddings)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: models/multilingual-e5-large
INFO:faiss.loader:Loading faiss with AVX512 support.
INFO:faiss.loader:Could not load library with AVX512 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx512'")
INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


In [7]:
sber_assistant = SimpleAssistantSber(prompt)
structured = JSONAssistantGPT(Terms)
gpt_assistant = SimpleAssistantGPT(prompt)

INFO:assistants.simple_assistants:Initializing chat model
INFO:assistants.simple_assistants:Initialized
INFO:assistants.json_assistants:Initializing model
INFO:assistants.json_assistants:set structured chain
INFO:assistants.json_assistants:Initialized
INFO:assistants.simple_assistants:Initializing chat model
INFO:assistants.simple_assistants:Initialized


In [8]:
def cleanup_text(text):
    return re.sub(r'##IMAGE##\s+\S+\.(png|jpg|jpeg|gif)', '', text)

In [ ]:
def generate_summary(refs):
    json_string = structured.ask_question(cleanup_text(refs))
    list_of_terms = json.loads(json_string)
    docs = []
    for term in list_of_terms['terms']:
        found = t_d_vs.similarity_search_with_relevance_scores(term['term'], k = 20)
        found = [doc.page_content for doc, score in found if score >= 0.78]
        #print(f'\n{term["term"]}:\n{found}\n')
        #found = sorted(found, key=lambda x: x.relevance, reverse=True)
        docs.extend(found)
    glossary_context = "\n\n".join(doc for doc in docs)
    #print(glossary_context)
    query = {
        "glossary": glossary_context,
        "query": cleanup_text(refs)
    }
    try:
        summary = sber_assistant.ask_question(query)
    except Exception as e:
        summary = gpt_assistant.ask_question(query)
    return summary

In [ ]:
df = pd.read_csv('./data/articles_data_summ.csv')
df = df[26:29]

In [11]:
print(df.columns)

Index(['no', 'systems', 'problem', 'solution', 'samples', 'links',
       'image_links', 'local_image_paths', 'refs', 'url'],
      dtype='object')


In [12]:
df['json_summary'] = df['refs'].apply(generate_summary)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ngw.devices.sberbank.ru:9443/api/v2/oauth "HTTP/1.1 200 OK"


nan: Группа 1С УХ

1С:УХ: информационная система Управление Холдингом

1С:УХ: – программное обеспечение 1С: Предприятие, предназначенное для автоматизации бизнес-процессов учета, планирования, контроля, мониторинга показателей и финансовых рисков

nan: Группа 1С СРМ

ОБ: оборудование

ОБ: Операционный блок

АХО: Хозяйственная группа

ТС: транспортное средство

ФСО: Федеральный стандарт оценки

DF: 1C:Docflow

УТН: устройство терминальное навигационное

РФ: Российская Федерация

1С:CRM: система для менеджмента бизнес-процессов

УД: Уральский дивизион

ФЛ: физическое лицо

ПР: прочее

БД: база данных

HR: Служба по работе с персоналом

ГПХ: гражданско-правовой характер

ФС: Филиальная сеть

Лимит: – предельная сумма финансирования, допустимая для
клиента/ продукта/ класса риска и т.п.

Лимит: – предельная сумма финансирования, допустимая для
клиента/ продукта/ класса риска и т.п.

РФ: Российская Федерация

ИР: информационный ресурс

ПЛ: предмет лизинга

ЛД: лизингодатель (ООО «Интерлизин

INFO:httpx:HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 402 Payment Required"
ERROR:assistants.simple_assistants:Unexpected error in ask_question: (URL('https://gigachat.devices.sberbank.ru/api/v1/chat/completions'), 402, b'{"status":402,"message":"Payment Required"}\n', Headers({'server': 'SynGX', 'date': 'Mon, 09 Dec 2024 07:08:10 GMT', 'content-type': 'application/json; charset=utf-8', 'content-length': '44', 'connection': 'keep-alive', 'access-control-allow-credentials': 'true', 'access-control-allow-headers': 'Origin, X-Requested-With, Content-Type, Accept, Authorization', 'access-control-allow-methods': 'GET, POST, DELETE, OPTIONS', 'access-control-allow-origin': 'https://beta.saluteai.sberdevices.ru', 'x-request-id': '3a1e2564-a8a6-4438-ba65-a9dd23b630e0', 'x-session-id': 'd449eb91-63ab-43b6-abfb-a83ba9715649', 'allow': 'GET, POST', 'strict-transport-security': 'max-age=31536000; includeSubDomains'}))
INFO:httpx:HTTP Request: POST htt

ПДЗ: просроченная дебиторская задолженность

График платежей: – согласованный лизингодателем и лизингополучателем порядок оплаты лизинговых платежей и выкупной цены, определенный в приложении к договору лизинга, с указанием размера каждого лизингового платежа, срока, до наступления которого каждый из лизинговых платежей должен быть уплачен лизингополучателем лизингодателю, с указанием размера выкупной цены и срока её оплаты лизингополучателем лизингодателю, с указанием суммы счетов-фактур, выставляемых лизингодателем лизингополучателю, периода их выставления, с указанием сумм аванса, зачитываемых в счет начисляемых услуг лизинга, и даты зачета

График снижения реализационной стоимости: – реализационная стоимость предмета лизинга, рассчитанная помесячно на весь срок лизинга

Структура графика лизинговых платежей: – сочетание основных параметров графика лизинговых платежей: аванс, срок лизинга, тип графика, коэффициент ускорения и тип ускорения, количество ступеней (для ступенчатого) и д

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 402 Payment Required"
ERROR:assistants.simple_assistants:Unexpected error in ask_question: (URL('https://gigachat.devices.sberbank.ru/api/v1/chat/completions'), 402, b'{"status":402,"message":"Payment Required"}\n', Headers({'server': 'SynGX', 'date': 'Mon, 09 Dec 2024 07:08:25 GMT', 'content-type': 'application/json; charset=utf-8', 'content-length': '44', 'connection': 'keep-alive', 'access-control-allow-credentials': 'true', 'access-control-allow-headers': 'Origin, X-Requested-With, Content-Type, Accept, Authorization', 'access-control-allow-methods': 'GET, POST, DELETE, OPTIONS', 'access-control-allow-origin': 'https://beta.saluteai.sberdevices.ru', 'x-request-id': 'b6c441d9-39fa-4f50-bd71-3943346a6fe0',

ЦБ РФ: Центральный банк Российской Федерации

РФ: Российская Федерация

ДДС: движение денежных средств

ДС: дополнительное соглашение

БКИ: бюро кредитных историй (НБКИ, ОКБ)

ТС: транспортное средство

БД: база данных

ДЛ: договор финансовой аренды (лизинга)

ДИ: движимое имущество

ЛД: лизингодатель (ООО «Интерлизинг»)

График снижения реализационной стоимости: – реализационная стоимость предмета лизинга, рассчитанная помесячно на весь срок лизинга

ДДС: движение денежных средств

ДЛ: договор финансовой аренды (лизинга)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
